In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, explode

In [3]:
spark = SparkSession.builder.appName("app").getOrCreate()
sc = spark.sparkContext

In [4]:
pathToRead = r"C:\Users\game_info.csv"

In [5]:
df = spark.read.csv(pathToRead,header=True,)

In [6]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- name: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- released: string (nullable = true)
 |-- tba: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- website: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: string (nullable = true)
 |-- playtime: string (nullable = true)
 |-- achievements_count: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- suggestions_count: string (nullable = true)
 |-- game_series_count: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- esrb_rating: string (nullable = true)
 |-- added_status_yet: string (nullable = true)
 |-- added_status_owned: string (nullable = true)
 |-- added_status_beaten: string

In [7]:
df1 = df.select(
    df["id"],
    df["slug"],
    df["name"],
    df["metacritic"],
    df["released"],
    df["tba"],
    df["updated"],
    df["website"],
    df["rating"],
    df["rating_top"],
    df["playtime"],
    df["achievements_count"],
    df["ratings_count"],
    df["suggestions_count"],
    df["game_series_count"],
    df["reviews_count"],
    split(col("platforms"), "\|\|").alias("platforms"),
    split(col("developers"), "\|\|").alias("developers"),
    split(col("genres"), "\|\|").alias("genres"),
    split(col("publishers"), "\|\|").alias("publishers"),
    df["esrb_rating"],
    df["added_status_yet"],
    df["added_status_owned"],
    df["added_status_beaten"],
    df["added_status_toplay"],
    df["added_status_dropped"],
    df["added_status_playing"],
)

In [8]:
df1.printSchema()

root
 |-- id: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- name: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- released: string (nullable = true)
 |-- tba: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- website: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: string (nullable = true)
 |-- playtime: string (nullable = true)
 |-- achievements_count: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- suggestions_count: string (nullable = true)
 |-- game_series_count: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- platforms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- developers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- publishers: array (nullable = true)
 |    |-- element: string (contai

In [57]:
pathToSave = r"C:\Users\game_info"
df.write.json(pathToSave)

In [9]:
df1.createOrReplaceTempView("games")

In [165]:
spark.sql("select name, platforms from games").show()

+--------------------+--------------------+
|                name|           platforms|
+--------------------+--------------------+
|     D/Generation HD|[PC, macOS, Xbox ...|
|G Prime Into The ...|[macOS, PC, Xbox ...|
|        Land Sliders|               [iOS]|
|          Pixel Gear| [PC, PlayStation 4]|
|      Gods and Idols|                [PC]|
|        Plague venue|               [Web]|
|The Moon Sliver (...|         [PC, macOS]|
|          Red Entity|  [PC, macOS, Linux]|
|       HippiesVsCops|                [PC]|
|They Came Through...|                [PC]|
|Fading Light: Des...|                [PC]|
|The Book Ritual (...|         [PC, macOS]|
|Corner of the Clu...|           [PC, Web]|
|Serious Sam Fusio...|  [Linux, macOS, PC]|
|Intelligence: The...|                [PC]|
|  Que Sera Sera Demo|         [PC, macOS]|
|Excelsior Phase O...|                [PC]|
|        Space Escape|               [Web]|
|Passing Through t...|               [Web]|
|           CORPUS100|          

# Which is the top most rated games accross all platform

In [172]:
df2 = df1.select(df["name"], df["released"], explode("platforms"), df["rating"], df["rating_top"], df["ratings_count"], df["playtime"])

In [173]:
df2.createOrReplaceTempView("filtered_games")

In [177]:
spark.sql("""
    select name, rating, platform from (
    select row_number() over(partition by col order by col) as num,
    name,
    col as platform,
    max(rating) over (partition by col) as rating
    from filtered_games
    where col != '0'
    order by rating desc) as table
    where num = 1
    """
).show()

+--------------------+------+----------------+
|                name|rating|        platform|
+--------------------+------+----------------+
|     D/Generation HD|   5.0|        Xbox One|
|     D/Generation HD|   5.0|              PC|
|     D/Generation HD|   5.0|   PlayStation 4|
|        Land Sliders|  4.86|             iOS|
|            Tron 2.0|  4.83|Game Boy Advance|
|Gobbo goes adventure|  4.83|         Android|
|     D/Generation HD|  4.78| Nintendo Switch|
|              Runbow|  4.78|           Wii U|
|            The Keep|  4.78|    Nintendo 3DS|
|Kane and Lynch: D...|  4.78|   PlayStation 3|
|     D/Generation HD|  4.78|           macOS|
|Royal Defense Ult...|  4.78|         PS Vita|
|          Red Entity|  4.78|           Linux|
|     Hogwarts Legacy|  4.73| Xbox Series S/X|
|Kane and Lynch: D...|  4.73|        Xbox 360|
|    Maximum Override|  4.73|   PlayStation 5|
| Destroy All Humans!|  4.71|             Wii|
|    FINAL FANTASY VI|  4.71|            SNES|
|    FINAL FA

# Which game dev has released most games

In [73]:
df3 = df1.select(df["name"], explode("developers"))

In [75]:
df3.createOrReplaceTempView("filtered_developers")

In [178]:
spark.sql("select col as developers, count(name) as games from filtered_developers group by col order by games desc limit 10").show()

+--------------------+-----+
|          developers|games|
+--------------------+-----+
|Sony Interactive ...|  647|
|                SEGA|  513|
|Konami Digital En...|  415|
|            Nintendo|  395|
|              Capcom|  395|
|      Big Fish Games|  380|
|     Electronic Arts|  316|
|             Ubisoft|  299|
|         Square Enix|  295|
|       Robert Brooks|  239|
+--------------------+-----+



# Which game genres has most games

In [91]:
df4 = df1.select(df["name"], explode("genres"))

In [92]:
df4.createOrReplaceTempView("filtered_genres")

In [98]:
spark.sql("select col as genres, count(name) as count from filtered_genres group by genres order by count desc limit 10").show()

+----------+------+
|    genres| count|
+----------+------+
|    Action|102023|
| Adventure| 72209|
|    Puzzle| 55550|
|Platformer| 48252|
|Simulation| 40719|
|     Indie| 33155|
|  Strategy| 33050|
|       RPG| 31947|
|   Shooter| 31822|
|    Casual| 28367|
+----------+------+

